In [ ]:
from selenium import webdriver
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from PIL import Image
import pytesseract
import time
from selenium.webdriver.support.ui import Select
import os
import pandas as pd
from selenium.common.exceptions import UnexpectedAlertPresentException, NoAlertPresentException
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
import html5lib
import chardet
import re

In [ ]:
#Aqui está montado o bloco que irá inicializar o navegador(mozilla)
def iniciar_navegador():
    servico = Service(GeckoDriverManager().install())
    navegador = webdriver.Firefox(service=servico)
    return navegador
#O bloco de acesso a página da IBRAPE
def acessar_pagina(navegador, url):
    navegador.get(url)
#bLOCO
def realizar_login(navegador, cpf, matricula, senha, nivel):
    WebDriverWait(navegador, 10).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="cpf"]'))
    ).send_keys(cpf)
    navegador.find_element(By.XPATH, '//*[@id="matricula"]').send_keys(matricula)
    navegador.find_element(By.XPATH, '//*[@id="senha"]').send_keys(senha)
    navegador.find_element(By.XPATH, '//*[@id="nivel"]').send_keys(nivel)

def resolver_captcha(navegador):
    captcha_image = navegador.find_element(By.XPATH, '/html/body/table/tbody/tr/td/table/tbody/tr/td/table/tbody/tr/td[3]/table/tbody/tr[2]/td/form/p[5]/img')
    captcha_image.screenshot('captcha.png')

    image = Image.open('captcha.png')
    captcha_text = pytesseract.image_to_string(image)

    captcha_input = navegador.find_element(By.XPATH, '//*[@id="security_code"]')
    captcha_input.send_keys(captcha_text)

def tentar_login_novamente(navegador, cpf, matricula, senha, nivel, url_sucesso):
    while True:
        try:
            # Preenche os campos de login e tenta resolver o CAPTCHA
            realizar_login(navegador, cpf, matricula, senha, nivel)
            resolver_captcha(navegador)
            
            # Clica no botão de login
            navegador.find_element(By.XPATH, '/html/body/table/tbody/tr/td/table/tbody/tr/td/table/tbody/tr/td[3]/table/tbody/tr[2]/td/form/p[6]/a/table/tbody/tr/td').click()
            
            time.sleep(4)  # Aguarda o redirecionamento pós-login
            
            # Verifica se o login foi bem-sucedido
            if navegador.current_url == url_sucesso:
                print("Login realizado com sucesso!")
                break
            else:
                print("Falha no login. Tentando novamente...")
        except UnexpectedAlertPresentException:
            try:
                # Captura e trata o alerta
                alerta = navegador.switch_to.alert
                print(f"Alerta detectado: {alerta.text}")
                alerta.dismiss()
            except NoAlertPresentException:
                print("Nenhum alerta presente.")
        except Exception as e:
            print(f"Ocorreu um erro inesperado: {e}")
        finally:
            time.sleep(2)  # Aguarda antes de tentar novamente

if __name__ == "__main__":
    url_login = "https://prefeituradeitaborai.online/engine.php?m=pref_login_fiscalizacao"
    url_sucesso = "https://prefeituradeitaborai.online/engine.php?m=pref_home_fiscal"
    cpf = "177.744.327-01"
    matricula = "48006"
    senha = "723447"
    nivel = "Nível 3"
    
    navegador = iniciar_navegador()
    acessar_pagina(navegador, url_login)
    tentar_login_novamente(navegador, cpf, matricula, senha, nivel, url_sucesso)
    
    # Continue com o restante do código após o login bem-sucedido

    # Navegar para a página de relatórios do sistema (BI)
    WebDriverWait(navegador, 10).until(
        EC.element_to_be_clickable((By.XPATH, '/html/body/table/tbody/tr/td/table/tbody/tr/td/table/tbody/tr/td[1]/table[2]/tbody/tr[31]/td'))
    ).click()
    
    WebDriverWait(navegador, 10).until(
        EC.element_to_be_clickable((By.XPATH, '/html/body/table/tbody/tr/td/table/tbody/tr/td/table/tbody/tr/td[1]/table[2]/tbody/tr[49]/td'))
    ).click()

    # Loop para extrair relatórios de 02/2022 até 07/2024
    mes_inicial = 2
    ano_inicial = 2022
    mes_final = 2
    ano_final = 2025

    ano_atual = ano_inicial
    mes_atual = mes_inicial

    while (ano_atual < ano_final) or (ano_atual == ano_final and mes_atual <= mes_final):
        competencia = f'{mes_atual:02d}/{ano_atual}'
        
        select_element = WebDriverWait(navegador, 10).until(
            EC.element_to_be_clickable((By.XPATH, '//*[@id="datainicialId"]'))
        )
        Select(select_element).select_by_visible_text(competencia)
        
        # Clicar para gerar o relatório
        gerar_relatorio = WebDriverWait(navegador, 10).until(
            EC.element_to_be_clickable((By.XPATH, '/html/body/table/tbody/tr/td/table/tbody/tr/td/table/tbody/tr/td[3]/table/tbody/tr[2]/td/div[1]/div[1]/form/div[2]/input'))
        )
        navegador.execute_script("arguments[0].click();", gerar_relatorio)
        time.sleep(15)
        
        # Clicar para baixar o relatório
        baixar_relatorio = WebDriverWait(navegador, 15).until(
            EC.element_to_be_clickable((By.XPATH, '/html/body/table/tbody/tr/td/table/tbody/tr/td/table/tbody/tr/td[3]/table/tbody/tr[2]/td/div[3]/form/button'))
        )
        navegador.execute_script("arguments[0].click();", baixar_relatorio)
        time.sleep(12)
        
        # Voltar para selecionar outra competência
        voltar_pagina = WebDriverWait(navegador, 15).until(
            EC.element_to_be_clickable((By.XPATH, '/html/body/table/tbody/tr/td/table/tbody/tr/td/table/tbody/tr/td[1]/table/tbody/tr[49]/td'))
        )
        navegador.execute_script("arguments[0].click();", voltar_pagina)
        time.sleep(10)
        
        mes_atual += 1
        if mes_atual > 12:
            mes_atual = 1
            ano_atual += 1

In [ ]:
#path = r'C:\Users\User\Documents\petrobrasnotas'

In [ ]:
#Direciona a pasta dos arquivos e altera o tipo do arquvi para HTML
path = r'C:\Users\User\OneDrive\Documentos\notaspetrobras'
files = os.listdir(path)


for index, file in enumerate(files):
    os.rename(os.path.join(path, file), os.path.join(path, ''.join([str(index), '.html'])))

In [ ]:
# with open(r'C:\Users\User\Documents\petrobrasnotas\0.html', 'r', encoding='utf-8') as f:
#     conteudo = f.read()

In [ ]:
# conteudo

In [ ]:
colunas = [
    'Emissor', 'Tomador', 'Nota', 'Dia', 'Mes/Ano', 'Vlr Nota', 'Deducao',
    'Base', 'ISS', 'Cancelada', 'Tributacao', 'Codigo', 'Descricao', 'Aliquota', 'DAM', 'Arquivo_Origem'
]

# Criar dicionário de conversores para forçar todas as colunas como strings
converters = {col: str for col in colunas}
converters

In [ ]:
# Diretório dos arquivos HTML
path = r"C:\Users\User\OneDrive\Documentos\notaspetrobras"

# Lista para armazenar DataFrames
dataFrames = []

# Iterar sobre os arquivos HTML na pasta
for arquivo in os.listdir(path):
    if arquivo.endswith('.html'):
        caminho_arquivo = os.path.join(path, arquivo)

        # Ler todas as tabelas do arquivo HTML
        tabelas = pd.read_html(caminho_arquivo, keep_default_na=False, converters=converters ) # Evita que valores vazios sejam convertidos para NaN

        # Adicionar todas as tabelas lidas à lista
        for df in tabelas:
            # Adicionar uma coluna com o nome do arquivo
            df['Arquivo_Origem'] = arquivo
            
            dataFrames.append(df)

# Concatenar todas as tabelas em um único DataFrame mantendo a exatidão dos dados
df_concatenado = pd.concat(dataFrames, ignore_index=True)


df_concatenado

In [ ]:
def limpar_numeros(df, colunas):
    """
    Remove pontos, vírgulas, letras e caracteres especiais das colunas especificadas,
    deixando apenas os números.

    Parâmetros:
    df (pd.DataFrame): O DataFrame contendo os dados.
    colunas (list): Lista de colunas para limpar.

    Retorna:
    pd.DataFrame: O DataFrame com as colunas limpas.
    """
    for coluna in colunas:
        if coluna in df.columns:
            # Aplica uma expressão regular para manter apenas os números
            df[coluna] = df[coluna].apply(lambda x: re.sub(r'[^0-9]', '', str(x)))
    return df


colunas_para_limpar = ['Vlr Nota', 'Deducao', 'Base', 'ISS']
df_limpo = limpar_numeros(df_concatenado, colunas_para_limpar)
df_limpo

In [ ]:
cnpjs_para_excluir = ['21.004.012/0001-64', '77.580.066/0001-22','37.249.433/0001-95','47.621.331/0001-02','19.411.731/0001-58', '12.216.767/0001-31', '11.898.201/0001-74',  '12.221.498/0001-00', '21.877.832/0001-60', '63.838.133/0001-51',  '45.738.773/0001-08', '97.624.678/0001-87', '33.918.233/0001-27',  '20.720.313/0001-21', '83.165.370/0001-06', '85.809.284/0001-14']

In [ ]:
# nota = 99000005
# # emissor = '15.563.826/0002-17'

In [ ]:
#df_filtrado = df_concatenado[(df_concatenado["Nota"] == nota) & (df_concatenado["Emissor"] == emissor)]

In [ ]:
# Filtrar o DataFrame, removendo os CNPJs indesejados
df_limpo = df_limpo[~df_limpo["Emissor"].isin(cnpjs_para_excluir)]

In [ ]:
mapeamento_dam = {
    ("15.563.826/0002-17", "33.000.167/0125-41", "99000021"): "105168",
    ("15.563.826/0002-17", "33.000.167/0125-41", "99000023"): "105168",
    ("16.273.641/0001-50", "33.000.167/0125-41", "99000023"): "105168",
    ("16.273.641/0001-50", "33.000.167/0125-41", "99000021"): "105168",
    ("00.516.634/0001-00", "33.000.167/0125-41", "99000001"): "110563",
    ("07.432.517/0001-07", "33.000.167/0125-41", "99000003"): "101397",
    ("07.432.517/0001-07", "33.000.167/0125-41", "99000004"): "101397",
    ("17.164.435/0001-74", "33.000.167/0125-41", "99000007"): "115636",
    ("03.560.020/0001-05", "33.000.167/0125-41", "99000103"): "110891",
    ("07.432.517/0001-07", "33.000.167/0125-41", "99000078"): "129513",
    ("07.356.428/0002-00", "33.000.167/0125-41", "99000006"): "129513",
    ("07.432.517/0001-07", "33.000.167/0125-41", "99000077"): "127022",
    ("16.273.641/0002-30", "33.000.167/0125-41", "99000007"): "129457",
    ("07.432.517/0001-07", "33.000.167/0125-41", "99000074"): "127022",
    ("07.432.517/0001-07", "33.000.167/0125-41", "99000073"): "127022",
    ("07.432.517/0001-07", "33.000.167/0125-41", "99000070"): "126110",
    ("07.432.517/0001-07", "33.000.167/0125-41", "99000071"): "126110",
    ("07.432.517/0001-07", "33.000.167/0125-41", "99000068"): "127042",
    ("07.432.517/0001-07", "33.000.167/0125-41", "99000066"): "125350",
    ("07.432.517/0001-07", "33.000.167/0125-41", "99000067"): "125350",
    ("07.432.517/0001-07", "33.000.167/0125-41", "99000065"): "126111",
    ("07.432.517/0001-07", "33.000.167/0125-41", "99000064"): "123874",
    ("07.432.517/0001-07", "33.000.167/0125-41", "99000062"): "123089",
    ("07.432.517/0001-07", "33.000.167/0125-41", "99000063"): "123089",
    ("07.432.517/0001-07", "33.000.167/0125-41", "99000061"): "123089",
    ("07.432.517/0001-07", "33.000.167/0125-41", "99000056"): "121162",
    ("07.432.517/0001-07", "33.000.167/0125-41", "99000057"): "121162",
}


mapeamento_canc = {
    ("15.563.826/0002-17", "33.000.167/0125-41", "99000080"): "SIM",
    ("15.063.096/0011-86", "33.000.167/0125-41", "99000024"): "SIM",
    ("15.563.826/0002-17", "33.000.167/0125-41", "99000109"): "SIM"
}


df_limpo = df_limpo.copy()
df_limpo["chave"] = list(zip(df_limpo["Emissor"], df_limpo["Tomador"], df_limpo["Nota"]))


In [ ]:
df_afetadas_dam = df_limpo[df_limpo["chave"].isin(mapeamento_dam.keys())].copy()
df_afetadas_canceladas = df_limpo[df_limpo["chave"].isin(mapeamento_canc.keys())].copy()

In [ ]:
print("\n🔹 Linhas ANTES da atualização (mapeamento_dam):")
df_afetadas_dam

In [ ]:
print("\n🔹 Linhas ANTES da atualização (notas canceladas):")
df_afetadas_canceladas

In [ ]:
# Atualizar a coluna "DAM"
df_limpo["DAM"] = df_limpo["chave"].map(mapeamento_dam).fillna(df_limpo["DAM"])

# Atualizar a coluna "Cancelada"
df_limpo["Cancelada"] = df_limpo["chave"].map(mapeamento_canc).fillna(df_limpo["Cancelada"])

In [ ]:
df_att = df_limpo[df_limpo["chave"].isin(mapeamento_dam.keys()) | df_limpo["chave"].isin(mapeamento_canc.keys())].copy()

print("\n✅ Linhas DEPOIS da atualização:")
df_att

In [ ]:
df_att.drop(columns=["chave"], inplace=True)

In [ ]:
df_att

In [ ]:
df_final = df_limpo.copy()
df_final

In [ ]:
novo_arquivo_path = r'C:\Users\User\Desktop\fIBRAPE.csv'
df_final.to_csv(novo_arquivo_path, index=False)

In [ ]:
novo_arquivo_path